# Transfer Learning

世の中にはたくさんの学習済みのモデルがあります。それらを使ってTransfer Learningを行います  
今回はtensorflow_hubにある学習済みモデルを使います。

## 学習済みモデルの再利用

まずはモデルを読み込み、使用してみます。

In [ ]:
!sudo pip install -q -U tf-hub-nightly

In [ ]:
import os
import shutil
import tensorflow as tf
import tensorflow_hub as hub

https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4  
今回はこのモデルを使用します。mobile_netは軽量で高速なモデルで、精度もそれなりに高く、非常に使いやすいモデルです。

In [ ]:
classifier_url ="https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
IMAGE_SHAPE = (224, 224)

classifier = tf.keras.Sequential([
    hub.KerasLayer(classifier_url, input_shape=IMAGE_SHAPE+(3,))
])

In [ ]:
!pip install pillow

試しに、好きな画像を学習済みモデルでpredictしてみましょう。

image_urlに好きな画像のurlを入れてみてください。

In [ ]:
import numpy as np
import PIL.Image as Image

image_url = "https://dol.ismcdn.jp/mwimgs/7/1/670m/img_71c53c1d81500a1cf73a4f543e72413f27838.jpg" # 自分で指定

img = tf.keras.utils.get_file('inu.jpg', image_url)
img = Image.open(img).resize(IMAGE_SHAPE)
img


In [ ]:
img = np.array(img) / 255.0
print(img.shape)

batch_sizeの分だけ次元を増やしてあげてから、predictしてみます。

In [ ]:
## <todo> これまでと同じようにexpandを使って、batch_size部分の次元を合わせます
## ヒント: expandする次元が今までと違うことに注意しましょう(1, 244, 244, 3)
img = ___
## <todo> classifierを使ってpredictを行います
## ヒント: https://keras.io/ja/models/sequential/ でpredictを行うメソッドを確認してください。
result = ___ 
## <todo> 出力のリストから一番確率の高い要素を出力してみましょう
## ヒント: https://numpy.org/doc/stable/reference/routines.sort.html から一番高い値のindexを返すメソッドを確認してください。
predicted_class = np.___(___, axis=-1) 
predicted_class

予測結果が得られたので、このclass_idがなんに紐づいているのか定義から確認してみます。

In [ ]:
labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())
predicted_class_name = imagenet_labels[predicted_class]
print(predicted_class_name)

きちんと学習済みで推論できていることが確認できます。 
他にも何枚か試してみてください。

ロードしたモデルについても確認しておきます。

In [ ]:
classifier.summary()

Trainable params: 0 からこのモデルが再トレーニングできないモデルなことがわかります。

## 最終層を再学習

学習済みモデルを利用することで、様々な物体を高精度に判別することができるモデルを、手軽に用意することが出来ました。  
しかしこのまま利用するには少し問題があります。再トレーニングができないので、このモデルで学習されていない画像が出た時に判別できないのです。

この問題を解決するために、Transfer Learning(転移学習)を行います。  
具体的には、最終層やいくつかの層をあえて取り外し、取り外した部分を再学習させるといったことを行います。  
これによって、学習済みモデルが持つ高精度な判別能力を維持しつつ、自分の問題設定に合わせた判別が可能となります。

では、実際に再学習させてみてTransfer Learningの効果を確認してみましょう。

### データセットの確認

まず、学習に使うデータセットを用意します。

https://www.tensorflow.org/datasets/catalog/overview  
上記のカタログの中から、  
https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet  
こちらのデータセットを今回は使います。

では、このデータセットをローカルに持ってきましょう。

In [ ]:
%%bash
DATASET_URL=https://www.robots.ox.ac.uk/~vgg/data/pets/data/images.tar.gz
curl -o images.tar.gz ${DATASET_URL}
tar -xf images.tar.gz

これから使うdataset apiに合わせるため、datasetフォルダの中にペットの種類毎のフォルダを作ってまとめておきます。

In [ ]:
DATASET_DIR = "dataset"
for image in os.listdir("images"):
    pet_kind = '_'.join(image.split("_")[:-1])
    os.makedirs(os.path.join(DATASET_DIR, pet_kind), exist_ok=True)
    if image.split(".")[-1] == 'jpg':
        shutil.copy("images/{}".format(image), os.path.join(DATASET_DIR, pet_kind))

dataset apiの一つである[tf.keras.preprocessing.image_dataset_from_directory](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image_dataset_from_directory?hl=en)を使ってデータセットを読み込みましょう。

In [ ]:
image_size = (224, 224)
batch_size = 32
DATASET_DIR = "dataset"

## <todo> ↑のURLを参考に___を埋めてみましょう。
## URL内のArgsに各引数の詳細が書かれているので、確認しましょう。
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=___,
    validation_split=0.2,
    subset=___,
    label_mode=___,
    seed=1111,
    image_size=image_size,
    batch_size=batch_size,
)
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    directory=___,
    validation_split=0.2,
    subset=___,
    label_mode=___,
    seed=1111,
    image_size=image_size,
    batch_size=batch_size,
)

読み込めたら、Fashion-MNISTと同じように画像データがどうなっているかを確認します。  
まず、データの形と、ラベルに何が存在するのかを確認してみましょう。

In [ ]:
for images, labels in val_data.take(1):
    print("Image shape (batch, height, width, channel): " + str(images.shape))
    print("Label shape (batch, classes): " + str(labels.shape))
    print("Pet classes: \n" + str(val_data.class_names))

バッチサイズや画像のサイズが指定した数になっていることが確認できると思います。  
また、Fashion-MNISTとは違い、今回のshapeには新たに画像のチャンネルが含まれています。  
ここから、データセットの画像がグレースケールではなく、RGBのカラー画像であることがわかります。

実際に画像を表示してみましょう。

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
def view_dataset():
    for images, labels in val_data.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title("label: " + val_data.class_names[np.argmax(labels[i])])
            plt.axis("off")
            
view_dataset()

犬や猫の犬種/猫種のカラー画像が表示できたかと思います。

データセット全体の分布も見てみましょう。

In [ ]:
class_names = val_data.class_names
left = range(0, len(class_names))
height = np.zeros(len(class_names))
for data in [train_data, val_data]:
    for _, y in data:
        height += np.sum(y, axis=0)
plt.xticks(rotation=90)
plt.bar(left, height, tick_label=class_names, align="center")

ほとんどのデータが約200枚であることが確認できました。

では、試しにこのデータを先ほどの学習済みモデル、classifierで判別してみましょう。

In [ ]:
plt.figure(figsize=(10, 10))
for images, labels in val_data.take(1):
    for i in range(3):
        ax = plt.subplot(3, 3, i + 1)
        norm_images = images / 255
        result = classifier.predict(norm_images)
        predicted_class = np.argmax(result[i], axis=-1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(
            "label: " + val_data.class_names[np.argmax(labels[i])] +
            ## <todo> ___を埋めてImageNetの予測ラベルを表示できるようにしてください
            ## ヒント: ImageNetのlabelのlistはimagenet_labelsで定義されています
            "\npredict: " + ___[___]
        )
        plt.axis("off")

何回か繰り返して画像を確認してみてください。  
すると、犬種/猫種を判別できるものと全く判別できないものがあると思います。  
純粋な判別ミスもありますが、ここで理解してもらいたいことは、`classifier(mobilenet)が学習に使用しているImageNetデータセットのクラスに存在しないものは判別できない`ということです。

今回だと
- Havanese
- Wheaten_terrier
- American_Bulldog
- American_Pit_Bull_Terrier
- Bombay 
- Bengal
- Rusian_Blue
- Ragdoll
- British_Shorthair
- Ragdoll
- Abysinian
- Sphynx
- ...

あたりはImageNetに含まれていないので判別できません。    
ImageNetにどういったクラスのものが存在するか興味がある方は、https://starpentagon.net/analytics/ilsvrc2012_class_image/
でチェックしてみると良いです。(公式では現在検索できなくなっているようです。)

こういったものも含めて犬種/猫種を判別できるようにするというのが、Transfer Learningの主目的となります。

### Preprocessing
dataset apiを使ったおかげで、今回はデータのshapeを変える必要はなさそうです。

[tf.keras.layers.experimental.preprocessing.Rescaling](https://www.tensorflow.org/api_docs/python/tf/keras/layers/experimental/preprocessing/Rescaling)を用いて正規化だけ行っておきましょう。

In [ ]:
## <todo> documentを参考に___を埋めてデータセットを正規化してください。
norm_layer = tf.keras.layers.experimental.preprocessing.Rescaling(scale=___)
norm_train_dataset = train_data.map(lambda x, y: (norm_layer(x), y))
norm_val_dataset = val_data.map(lambda x, y: (norm_layer(x), y))

### モデルの構築
データを用意することができたので、学習に使用するモデルを構築します。

今回はTransfer Learningの有無による精度の差を確認したいので、  
Fashion-MNISTで使った通常のCNNモデルと、Transfer Learningするモデルの二つを構築していきます。

まず、CNNのモデルを定義しましょう。

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

## <todo> データセットのクラス数分の値を___に入れてください。
## ヒント: クラス名はデータセットのclass_names属性にあるので、そのlenをとることでクラス数分の値を取得できそうです。
NUM_CLASSES =　___

def cnn():
    model = tf.keras.Sequential()
    model.add(Conv2D(32, kernel_size = (3,3), activation = "relu"))
    model.add(Conv2D(64, kernel_size = (3,3), activation = "relu"))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Conv2D(64, kernel_size = (3,3), activation = "relu"))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(128, activation="relu"))
    model.add(Dropout(0.5))
    model.add(Dense(NUM_CLASSES, activation = "softmax"))
    return model

cnn_model = cnn()

次に、Transfer Learningするモデルを構築します。
特徴量ベクトルが取り出すことができる学習済みモデルを読み込みます。

https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4

ここからurlをコピーして使います。

In [ ]:
feature_vector_url = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4"

IMAGE_SHAPE = (224, 224)

def mobilenet_v2():
    model = tf.keras.Sequential([
        hub.KerasLayer(feature_vector_url, input_shape=IMAGE_SHAPE+(3,)),
        ## <todo> クラス数分のpredictが出来るように最終層にDenseを追加してください。
        ## ヒント: これまでと同じような形でDence層のメソッドを追加すれば良いです。出力層になるので、unit数と活性化関数に注意してください。
    ])
    return model

transfer_learning_model = mobilenet_v2()
transfer_learning_model.summary()

最終層につけたDenseがTrainableで、特徴量ベクトルの層がUntrainableになっていることが確認できるかと思います。  
このTrainable部分を学習させることで、学習モデルにはない犬種/猫種も判別できるモデルにすることができます。

### Training

モデルの構築ができたので、訓練を開始します。

In [ ]:
## <todo> ___を埋めて最適化関数がadam, 損失関数が'categorical_crossentropy'モデルをコンパイルできるようにしてください
cnn_model.compile(
  ___,
  ___,
  metrics=['categorical_accuracy']
)

In [ ]:
## <todo> 同じように___を埋めて最適化関数がadam, 損失関数が'categorical_crossentropyのモデルをコンパイルできるようにしてください
transfer_learning_model.compile(
  ___,
  ___,
  metrics=['categorical_accuracy']
)

https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/Callback
を参考に、指標として使いたいlossとaccuracyをbatch毎(step毎)に記録します。

In [ ]:
class CollectBatchStats(tf.keras.callbacks.Callback):
    def __init__(self):
        self.batch_losses = []
        self.batch_acc = []
        self.epoch_val_losses = []
        self.epoch_val_acc = []
        
    def on_train_batch_end(self, batch, logs=None):
        self.batch_losses.append(logs['loss'])
        self.batch_acc.append(logs['categorical_accuracy'])
        self.model.reset_metrics()
        
    def on_epoch_end(self, epoch, logs=None):
        ##  <todo> 対応するリストに'val_loss'と'val_categorical_accuracy'のlogを追加してください
        ## ヒント: 一つ上のメソッド内を参考にしてください
        
        self.model.reset_metrics()

In [ ]:
batch_stats_callback_cnn = CollectBatchStats()
## <todo> ___に前処理後のtrainとvalidationのデータセットをいれて学習が開始できるようにしてください
## ヒント: 前処理をしているセルを確認してみてください。
cnn_model.fit(___, validation_data=___, epochs=5, callbacks=[batch_stats_callback_cnn])

In [ ]:
batch_stats_callback_transfer_learning = CollectBatchStats()
## <todo>  同じように___に前処理後のtrainとvalidationのデータセットをいれて学習が開始できるようにしてください
## ヒント: 前処理をしているセルを確認してみてください。
transfer_learning_model.fit(___, validation_data=___, epochs=5, callbacks=[batch_stats_callback_transfer_learning])

### 可視化
訓練が終わったら、各モデルのlossとaccuracyの推移を確認してみます。

In [ ]:
plt.figure(figsize=(10, 7))
plt.suptitle("Training", fontsize=15)
plt.subplot(2, 2, 1)
plt.title('Model loss')
plt.ylabel("Loss")
plt.xlabel("Steps")
plt.ylim([0,4])
plt.plot(batch_stats_callback_cnn.batch_losses, label="CNN model")
plt.plot(batch_stats_callback_transfer_learning.batch_losses, label="Transfer Learning model")
plt.grid()

plt.subplot(2, 2, 2)
plt.title('Model accuracy')
plt.ylabel("Accuracy")
plt.xlabel("Steps")
plt.ylim([0,1])
plt.plot(batch_stats_callback_cnn.batch_acc, label="CNN model")
plt.plot(batch_stats_callback_transfer_learning.batch_acc, label="Transfer Learning model")
plt.grid()

plt.legend(bbox_to_anchor=[1, 1])

In [ ]:
plt.figure(figsize=(10, 7))
plt.suptitle("Validation", fontsize=15)
plt.subplot(2, 2, 1)
plt.title('Model loss')
plt.ylabel("Loss")
plt.xlabel("Steps")
plt.plot(batch_stats_callback_cnn.epoch_val_losses, label="CNN model")
plt.plot(batch_stats_callback_transfer_learning.epoch_val_losses, label="Transfer Learning model")
plt.grid()

plt.subplot(2, 2, 2)
plt.title('Model accuracy')
plt.ylabel("Accuracy")
plt.xlabel("Steps")
plt.plot(batch_stats_callback_cnn.epoch_val_acc, label="CNN model")
plt.plot(batch_stats_callback_transfer_learning.epoch_val_acc, label="Transfer Learning model")
plt.grid()

plt.legend(bbox_to_anchor=[1, 1])

両モデルの違いを確認できたでしょうか。

各モデルの個別の予測結果も確認しましょう。  
先程作ったview_datasetメソッドを改良して、ラベルの他にpredictも出力できるようにしましょう。

In [ ]:
def view_dataset(model=None):
    for images, labels in val_data.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            if not model:
                plt.title("label: " + val_data.class_names[np.argmax(labels[i])])
            else:
                norm_img = images / 255
                result = model.predict(norm_img)
                plt.title(
                    "label:" + val_data.class_names[np.argmax(labels[i])] +
                    ## <todo> ___を埋めてmodelがpredictした予測ラベルを表示できるようにしましょう
                    ## 上の"label:"でやってくことが参考になります。predictした結果を渡すことに注意しましょう
                    "\npredict:" + ______[___(___[_])]
                )
            plt.axis("off")

In [ ]:
plt.figure(figsize=(10, 10))
view_dataset(cnn_model)

In [ ]:
plt.figure(figsize=(10, 10))
view_dataset(transfer_learning_model)

ImageNetデータセットにない犬種/猫種も予測できたでしょうか。